In [1]:
import os, shutil
import random
import tensorflow as tf
from tensorflow.keras import (models, layers, optimizers)

import plotly.express as ex
import plotly.graph_objects as go

2023-01-24 20:55:26.924081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 20:55:27.188897: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 20:55:27.232775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 20:55:27.232795: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
data_path = "../data/raw"
x_train = os.listdir(data_path + "/train")
# x_test = os.listdir(data_path + "/test1")
x_train = [os.path.join(data_path, "train", x) for x in x_train]
# x_test = [os.path.join(data_path, "test1", x) for x in x_test]
print(len(x_train))
# print(len(x_test))
print(x_train[:5])
# print(x_test[:5])

25000
['../data/raw/train/dog.11082.jpg', '../data/raw/train/cat.2955.jpg', '../data/raw/train/cat.12060.jpg', '../data/raw/train/dog.971.jpg', '../data/raw/train/cat.6966.jpg']


In [3]:
x_train = random.choices(x_train, k=3000)
# x_test = random.choices(x_train, k=250)
print(len(x_train))
# print(len(x_test))

3000


In [4]:
data_path = "../data/processed"
if os.path.exists(data_path):
    shutil.rmtree(data_path)
os.makedirs(data_path + "/train/dogs")
os.makedirs(data_path + "/train/cats")

In [5]:
for x in x_train:
    if "dog" in x:
        shutil.copy(x, data_path + "/train/dogs")
    else:
        shutil.copy(x, data_path + "/train/cats")

In [6]:
print(len(os.listdir(data_path + "/train/dogs")))
print(len(os.listdir(data_path + "/train/cats")))

1424
1382


In [7]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

2023-01-24 20:55:31.100663: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-24 20:55:31.100971: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-24 20:55:31.101027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hope): /proc/driver/nvidia/version does not exist
2023-01-24 20:55:31.102105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

In [9]:
batch_size = 16
x_train, x_valid = tf.keras.utils.image_dataset_from_directory(
    (data_path + "/train"),
    batch_size=batch_size,
    image_size=(150, 150),
    validation_split=.2,
    subset="both",
    seed=42
)

Found 2806 files belonging to 2 classes.
Using 2245 files for training.
Using 561 files for validation.


In [10]:
x_train_count = len(x_train) * batch_size
x_valid_count = len(x_valid) * batch_size
print(x_train_count)
print(x_valid_count)

2256
576


In [11]:
history = model.fit(
    x_train.repeat(),
    steps_per_epoch=int(x_train_count/batch_size),
    epochs=25,
    validation_data=x_valid.repeat(),
    validation_steps=int(x_valid_count/batch_size)
)

Epoch 1/25
141/141 [==============================] - 20s 139ms/step - loss: 1.4787 - acc: 0.5621 - val_loss: 0.7340 - val_acc: 0.5365
Epoch 2/25
141/141 [==============================] - 19s 138ms/step - loss: 0.6566 - acc: 0.6441 - val_loss: 0.8176 - val_acc: 0.5597
Epoch 3/25
141/141 [==============================] - 19s 133ms/step - loss: 0.5469 - acc: 0.7261 - val_loss: 0.7938 - val_acc: 0.6043
Epoch 4/25
141/141 [==============================] - 19s 138ms/step - loss: 0.4381 - acc: 0.7933 - val_loss: 0.9934 - val_acc: 0.5740
Epoch 5/25
141/141 [==============================] - 19s 135ms/step - loss: 0.3659 - acc: 0.8272 - val_loss: 0.7608 - val_acc: 0.6827
Epoch 6/25
141/141 [==============================] - 19s 138ms/step - loss: 0.2722 - acc: 0.8811 - val_loss: 0.9747 - val_acc: 0.6292
Epoch 7/25
141/141 [==============================] - 20s 140ms/step - loss: 0.1959 - acc: 0.9198 - val_loss: 0.9210 - val_acc: 0.6702
Epoch 8/25
141/141 [==============================] - 2

In [ ]:
acc = history.history['acc']
vall_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=acc, name="Training Accuracy"))
fig.add_trace(go.Scatter(x=epochs, y=vall_acc, name="Validation Accuracy"))
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy"
)
fig.show()